In [65]:
import numpy as np
import pandas as pd
import os, sys
import codecs
import utm
from pyproj import Proj, transform
from scipy.io import savemat

In [66]:
cwd = os.getcwd()

In [67]:
directory_name = "/Veneto/"
file_name_1 = "Sotterranee_livello_piezometrico_PD_1999_2021_us"
file_name_2 = "anagrafica_" + file_name_1
original_file_type = ".csv"
#change last part depending on imported file type
file_path_1 = cwd + directory_name + "/format_1/" + file_name_1 +  original_file_type
file_path_2 = cwd + directory_name + "/format_2/" + file_name_2  + original_file_type
df_1 = pd.read_csv(file_path_1, sep=',', encoding = 'iso-8859-1')
df_2 = pd.read_csv(file_path_2, sep=',', encoding = 'iso-8859-1')
#df = df.drop('Unnamed: 0', axis = 1)
df = df_1.merge(df_2, on = "NPOZZO")

In [68]:
whole_table = pd.DataFrame()

for files in os.listdir(cwd + directory_name + "/format_1"):
    if files.endswith("us" + original_file_type):
        
        if not (os.path.exists(cwd + directory_name + "/combined_tables")): 
            os.makedirs(cwd + directory_name + "/combined_tables")
        
        file_path = cwd + directory_name + "format_1/" + files
        
        #if original file type isn't excel change pd.read_excel
        data = pd.read_csv(file_path)
        
        file_path_1 = cwd + directory_name + "format_1/" + files
        file_path_2 = cwd + directory_name + "format_2/" + "anagrafica_" + files
        df_1 = pd.read_csv(file_path_1, sep=',', encoding = 'iso-8859-1')
        df_2 = pd.read_csv(file_path_2, sep=',', encoding = 'iso-8859-1')
        #df = df.drop('Unnamed: 0', axis = 1)
        df = df_1.merge(df_2, on = "NPOZZO")
        
        whole_table = whole_table.append(df)

        #If you don't want an .xlsx file you can change data.to_excel
        df.to_excel(cwd + "/Veneto/combined_tables/" + files[:-4] + ".xlsx", index = False)

In [69]:
whole_table = whole_table[whole_table['LIVELLOSTATICOmslm'] > -100]
hold_table = pd.DataFrame()
for i in whole_table['NPOZZO'].unique():
    hold_table = hold_table.append(whole_table[whole_table['NPOZZO'] == i].head(1))
hold_table

,Unnamed: 0_x,NPOZZO,DATA,LIVELLOSTATICOmslm,LIVELLODAPRm,NOTELIVELLO,Unnamed: 0_y,X_GBO,Y_GBO,COMUNE,PROV,QUOTAPRmslm,QUOTAPCmslm,FALDA,PROFONDITA,DATIda,DATIa,unnamed
2,2,9,08/11/2000,32.060,-35.74,-,0,1734489.246,5067576.560,VEDELAGO,TV,67.800,67.05,falda libera,35.80,2000.0,2009.0,NaN
39,39,23,05/05/1999,31.330,-53.42,-,1,1729759.123,5070504.575,ALTIVOLE,TV,84.750,86.38,falda libera,85.97,1999.0,2020.0,NaN
124,124,31,25/05/1999,29.600,-27.20,-,2,1750864.569,5074892.367,ARCADE,TV,56.800,58.50,falda libera,56.20,1999.0,2009.0,NaN
166,166,36,03/05/1999,-1.830,-2.60,-,3,1768556.404,5051330.813,RONCADE,TV,0.770,0.00,falda libera,5.70,1999.0,2021.0,NaN
259,259,52,09/11/2000,30.360,-50.04,-,4,1738820.784,5069781.172,TREVIGNANO,TV,80.400,80.00,falda libera,40.00,2000.0,2009.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,2081,919,30/03/2006,3.650,-2.10,-,31,1708522.000,4993031.000,VILLANOVA DEL GHEBBO,RO,5.750,5.26,falda libera,12.00,2006.0,2021.0,NaN
2145,2145,920,30/03/2006,3.650,-2.15,-,32,1708522.000,4993031.000,VILLANOVA DEL GHEBBO,RO,5.800,5.28,falda confinata,17.00,2006.0,2021.0,NaN
2207,2207,921,30/03/2006,2.800,-2.03,-,33,1712792.000,4988086.000,VILLAMARZANA,RO,4.830,4.32,falda libera,6.30,2006.0,2021.0,NaN
2271,2271,922,30/03/2006,2.140,-2.69,-,34,1712792.000,4988086.000,VILLAMARZANA,RO,4.830,4.33,falda confinata,19.00,2006.0,2021.0,NaN


In [70]:
static_table = pd.DataFrame()

for i in whole_table['NPOZZO'].unique():
    hold = whole_table[whole_table['NPOZZO'] == i].head(1)
    inProj = Proj(init='epsg:3003');
    outProj = Proj(init='epsg:4326');
    x1,y1 = hold['X_GBO'], hold['Y_GBO'];
    x2,y2 = transform(inProj,outProj,x1,y1);
    hold['Latitude'] = x2
    hold['Longitude'] = y2

    static_table = static_table.append(hold)
    
static_table

/Users/andrew/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/andrew/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/m_/zzpt2tzj0n59x8crx_z4dn1w0000gn/T/ipykernel_41178/2148602337.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgra

,Unnamed: 0_x,NPOZZO,DATA,LIVELLOSTATICOmslm,LIVELLODAPRm,NOTELIVELLO,Unnamed: 0_y,X_GBO,Y_GBO,COMUNE,PROV,QUOTAPRmslm,QUOTAPCmslm,FALDA,PROFONDITA,DATIda,DATIa,unnamed,UTM_Lat,UTM_Long
2,2,9,08/11/2000,32.060,-35.74,-,0,1734489.246,5067576.560,VEDELAGO,TV,67.800,67.05,falda libera,35.80,2000.0,2009.0,NaN,12.012867,45.721827
39,39,23,05/05/1999,31.330,-53.42,-,1,1729759.123,5070504.575,ALTIVOLE,TV,84.750,86.38,falda libera,85.97,1999.0,2020.0,NaN,11.953555,45.749732
124,124,31,25/05/1999,29.600,-27.20,-,2,1750864.569,5074892.367,ARCADE,TV,56.800,58.50,falda libera,56.20,1999.0,2009.0,NaN,12.226757,45.781831
166,166,36,03/05/1999,-1.830,-2.60,-,3,1768556.404,5051330.813,RONCADE,TV,0.770,0.00,falda libera,5.70,1999.0,2021.0,NaN,12.440930,45.563496
259,259,52,09/11/2000,30.360,-50.04,-,4,1738820.784,5069781.172,TREVIGNANO,TV,80.400,80.00,falda libera,40.00,2000.0,2009.0,NaN,12.069534,45.740161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,2081,919,30/03/2006,3.650,-2.10,-,31,1708522.000,4993031.000,VILLANOVA DEL GHEBBO,RO,5.750,5.26,falda libera,12.00,2006.0,2021.0,NaN,11.648065,45.059868
2145,2145,920,30/03/2006,3.650,-2.15,-,32,1708522.000,4993031.000,VILLANOVA DEL GHEBBO,RO,5.800,5.28,falda confinata,17.00,2006.0,2021.0,NaN,11.648065,45.059868
2207,2207,921,30/03/2006,2.800,-2.03,-,33,1712792.000,4988086.000,VILLAMARZANA,RO,4.830,4.32,falda libera,6.30,2006.0,2021.0,NaN,11.700145,45.014134
2271,2271,922,30/03/2006,2.140,-2.69,-,34,1712792.000,4988086.000,VILLAMARZANA,RO,4.830,4.33,falda confinata,19.00,2006.0,2021.0,NaN,11.700145,45.014134


In [71]:
static_table

,Unnamed: 0_x,NPOZZO,DATA,LIVELLOSTATICOmslm,LIVELLODAPRm,NOTELIVELLO,Unnamed: 0_y,X_GBO,Y_GBO,COMUNE,PROV,QUOTAPRmslm,QUOTAPCmslm,FALDA,PROFONDITA,DATIda,DATIa,unnamed,UTM_Lat,UTM_Long
2,2,9,08/11/2000,32.060,-35.74,-,0,1734489.246,5067576.560,VEDELAGO,TV,67.800,67.05,falda libera,35.80,2000.0,2009.0,NaN,12.012867,45.721827
39,39,23,05/05/1999,31.330,-53.42,-,1,1729759.123,5070504.575,ALTIVOLE,TV,84.750,86.38,falda libera,85.97,1999.0,2020.0,NaN,11.953555,45.749732
124,124,31,25/05/1999,29.600,-27.20,-,2,1750864.569,5074892.367,ARCADE,TV,56.800,58.50,falda libera,56.20,1999.0,2009.0,NaN,12.226757,45.781831
166,166,36,03/05/1999,-1.830,-2.60,-,3,1768556.404,5051330.813,RONCADE,TV,0.770,0.00,falda libera,5.70,1999.0,2021.0,NaN,12.440930,45.563496
259,259,52,09/11/2000,30.360,-50.04,-,4,1738820.784,5069781.172,TREVIGNANO,TV,80.400,80.00,falda libera,40.00,2000.0,2009.0,NaN,12.069534,45.740161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,2081,919,30/03/2006,3.650,-2.10,-,31,1708522.000,4993031.000,VILLANOVA DEL GHEBBO,RO,5.750,5.26,falda libera,12.00,2006.0,2021.0,NaN,11.648065,45.059868
2145,2145,920,30/03/2006,3.650,-2.15,-,32,1708522.000,4993031.000,VILLANOVA DEL GHEBBO,RO,5.800,5.28,falda confinata,17.00,2006.0,2021.0,NaN,11.648065,45.059868
2207,2207,921,30/03/2006,2.800,-2.03,-,33,1712792.000,4988086.000,VILLAMARZANA,RO,4.830,4.32,falda libera,6.30,2006.0,2021.0,NaN,11.700145,45.014134
2271,2271,922,30/03/2006,2.140,-2.69,-,34,1712792.000,4988086.000,VILLAMARZANA,RO,4.830,4.33,falda confinata,19.00,2006.0,2021.0,NaN,11.700145,45.014134


In [72]:
static_table = static_table[['NPOZZO', 'UTM_Lat', 'UTM_Long', 'COMUNE', 'PROV', 'QUOTAPCmslm', 'QUOTAPRmslm', 'FALDA', 'PROFONDITA', 'DATIda', 'DATIa']]

In [73]:
if not (os.path.exists(cwd + "/processed_data/Veneto_Processed")): 
            os.makedirs(cwd + "/processed_data/Veneto_Processed")
        
        
combine_table = whole_table[['NPOZZO', 'DATA', 'LIVELLOSTATICOmslm', 'LIVELLODAPRm']]
combine_table.to_excel(cwd + "/processed_data/Veneto_Processed/COMBINED_VENETO_TABLE.xlsx", index = False)

In [74]:
# for i in whole_table['NPOZZO'].unique():
#     print(str(whole_table[whole_table['NPOZZO'] == i]['DATIda'].min())[:4])

In [75]:
whole_table

,Unnamed: 0_x,NPOZZO,DATA,LIVELLOSTATICOmslm,LIVELLODAPRm,NOTELIVELLO,Unnamed: 0_y,X_GBO,Y_GBO,COMUNE,PROV,QUOTAPRmslm,QUOTAPCmslm,FALDA,PROFONDITA,DATIda,DATIa,unnamed
2,2,9,08/11/2000,32.060,-35.74,-,0,1734489.246,5067576.56,VEDELAGO,TV,67.800,67.05,falda libera,35.8,2000.0,2009.0,NaN
3,3,9,07/02/2001,31.900,-35.90,-,0,1734489.246,5067576.56,VEDELAGO,TV,67.800,67.05,falda libera,35.8,2000.0,2009.0,NaN
5,5,9,02/08/2001,32.720,-35.08,-,0,1734489.246,5067576.56,VEDELAGO,TV,67.800,67.05,falda libera,35.8,2000.0,2009.0,NaN
9,9,9,26/07/2002,32.400,-35.40,-,0,1734489.246,5067576.56,VEDELAGO,TV,67.800,67.05,falda libera,35.8,2000.0,2009.0,NaN
17,17,9,18/08/2004,33.030,-34.77,-,0,1734489.246,5067576.56,VEDELAGO,TV,67.800,67.05,falda libera,35.8,2000.0,2009.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,2376,923,22/10/2020,-4.018,-3.30,-,35,1752440.920,4994083.18,LOREO,RO,-0.718,NaN,falda libera,7.0,2010.0,2021.0,NaN
2377,2377,923,25/01/2021,-3.718,-3.00,-,35,1752440.920,4994083.18,LOREO,RO,-0.718,NaN,falda libera,7.0,2010.0,2021.0,NaN
2378,2378,923,08/04/2021,-3.768,-3.05,-,35,1752440.920,4994083.18,LOREO,RO,-0.718,NaN,falda libera,7.0,2010.0,2021.0,NaN
2379,2379,923,12/07/2021,-3.718,-3.00,-,35,1752440.920,4994083.18,LOREO,RO,-0.718,NaN,falda libera,7.0,2010.0,2021.0,NaN


In [76]:
for i in whole_table['NPOZZO'].unique():
    hold = whole_table[whole_table['NPOZZO'] == i]
    max_year = str(whole_table[whole_table['NPOZZO'] == i]['DATIa'].max())[:4]
    min_year = str(whole_table[whole_table['NPOZZO'] == i]['DATIda'].min())[:4]
    hold = hold[['NPOZZO', 'DATA', 'LIVELLOSTATICOmslm', 'LIVELLODAPRm']]
    hold.to_excel(cwd + "/processed_data/Veneto_Processed/" + str(i) + "_" +
                  min_year + "_" + 
                  max_year +
                  ".xlsx", index = False)
    if not (os.path.exists(cwd + "/processed_data/Veneto_Processed/mat_files/")): 
            os.makedirs(cwd + "/processed_data/Veneto_Processed/mat_files/")
    savemat(cwd + "/processed_data/Veneto_Processed/mat_files/" + str(i) + "_" +
                  min_year + "_" + 
                  max_year + ".mat", hold)

In [77]:
static_table

,NPOZZO,UTM_Lat,UTM_Long,COMUNE,PROV,QUOTAPCmslm,QUOTAPRmslm,FALDA,PROFONDITA,DATIda,DATIa
2,9,12.012867,45.721827,VEDELAGO,TV,67.05,67.800,falda libera,35.80,2000.0,2009.0
39,23,11.953555,45.749732,ALTIVOLE,TV,86.38,84.750,falda libera,85.97,1999.0,2020.0
124,31,12.226757,45.781831,ARCADE,TV,58.50,56.800,falda libera,56.20,1999.0,2009.0
166,36,12.440930,45.563496,RONCADE,TV,0.00,0.770,falda libera,5.70,1999.0,2021.0
259,52,12.069534,45.740161,TREVIGNANO,TV,80.00,80.400,falda libera,40.00,2000.0,2009.0
...,...,...,...,...,...,...,...,...,...,...,...
2081,919,11.648065,45.059868,VILLANOVA DEL GHEBBO,RO,5.26,5.750,falda libera,12.00,2006.0,2021.0
2145,920,11.648065,45.059868,VILLANOVA DEL GHEBBO,RO,5.28,5.800,falda confinata,17.00,2006.0,2021.0
2207,921,11.700145,45.014134,VILLAMARZANA,RO,4.32,4.830,falda libera,6.30,2006.0,2021.0
2271,922,11.700145,45.014134,VILLAMARZANA,RO,4.33,4.830,falda confinata,19.00,2006.0,2021.0


In [78]:
static_table.to_excel(cwd + "/processed_data/Veneto_Processed/STATIC_VENETO_TABLE.xlsx", index = False)